In [2]:
#test

import torch
from torch import nn
from utils import *
import torch.nn.functional as F
from math import sqrt
from itertools import product as product
import torchvision 
# torchvision은 computer vision에 쓰이는 유명한 데이터셋 또는 모델구조로 이루어진라이브러리

input_image=torch.randn(1,3,300,300,dtype=torch.float)
print(input_image.shape)
conv1=nn.Conv2d(3,64,3,1)
m_pool1=nn.MaxPool2d(3,2)

featuremap1=conv1(input_image)
print(featuremap1)

featuremap2=m_pool1(featuremap1)
print(featuremap2)

torch.Size([1, 3, 300, 300])
tensor([[[[ 0.2208, -0.2578, -0.1035,  ..., -0.4964,  0.0062,  0.5476],
          [ 0.2477,  0.1464, -0.2509,  ...,  0.1487,  0.0274, -0.1243],
          [-0.3696,  0.3553, -0.1572,  ...,  0.9139,  0.6458, -0.0987],
          ...,
          [ 0.0665, -0.1157, -0.4980,  ...,  0.2773,  0.3946, -0.4749],
          [ 0.1275, -0.1993,  0.4204,  ..., -0.1676, -0.7198,  0.3852],
          [ 0.6056, -0.3908,  0.1105,  ..., -0.0161,  0.2145, -0.4443]],

         [[ 0.2489,  0.5186,  0.0571,  ..., -0.2068, -0.2171,  0.2488],
          [-0.1958, -0.3270,  0.1815,  ...,  0.6893, -0.1470, -0.6674],
          [-0.4262,  0.1253, -0.3114,  ...,  0.8007,  0.3910, -0.4102],
          ...,
          [-0.0947, -0.0770,  0.6597,  ...,  0.4777, -0.7171, -0.1634],
          [ 0.4109, -0.0555,  0.4642,  ..., -0.6372, -0.1219, -0.7765],
          [-0.4584, -0.4333, -0.6011,  ..., -0.1299, -0.0635,  0.3250]],

         [[-0.2038, -1.1173,  0.6671,  ...,  0.5935, -0.0977, -0.8671],
 

In [15]:
class SSD(nn.Module):
    def _init_(self):
        self.conv1_1=nn.Conv2d(3,64,3,padding=1) # stride는 default로 1로 지정되어있음
        self.conv1_2=nn.Conv2d(64,64,3,padding=1)
        self.pool1=nn.MaxPool2d(2,2)
        
        self.conv2_1=nn.Conv2d(64,128,3,padding=1)
        self.conv2_2=nn.Conv2d(128,128,3,padding=1)
        self.pool2=nn.MaxPool2d(2,2)
        
        self.conv3_1=nn.Conv2d(128,256,3,padding=1)
        self.conv3_2=nn.Conv2d(256,256,3,padding=1)
        self.conv3_3=nn.Conv2d(256,256,3,padding=1)
        self.pool3=nn.MaxPool2d(2,2,ceil_mode=True)
        
        self.conv4_1=nn.Conv2d(256,512,3,padding=1)
        self.conv4_2=nn.Conv2d(512,512,3,padding=1)
        self.conv4_3=nn.Conv2d(512,512,3,padding=1)
        self.pool4=nn.MaxPool2d(2,2)
        
        self.conv5_1=nn.Conv2d(512,512,3,padding=1)
        self.conv5_2=nn.Conv2d(512,512,3,padding=1)
        self.conv5_3=nn.Conv2d(512,512,3,padding=1)
        self.pool5=nn.MaxPool2d(3,1,1)
        
        self.conv6=nn.Conv2d(512, 1024, kernel_size=3, padding=1) #padding=6, dilation=6
        self.conv7=nn.Conv2d(1024,1024,3,padding=1)
        
        self.conv8_1=nn.Conv2d(1024,256,1,padding=0)
        self.conv8_2=nn.Conv2d(256,512,3,padding=1,stride=2)
        
        self.conv9_1=nn.Conv2d(512,128,1,padding=1)
        self.conv9_2=nn.Conv2d(128,256,3,padding=0,stride=2)
        
        self.conv10_1=nn.Conv2d(256,128,1,padding=0)
        self.conv10_2=nn.Conv2d(128,256,3,padding=0,stride=1)
        
        self.conv11_1=nn.Conv2d(256,128,1,padding=0)
        self.conv11_2=nn.Conv2d(128,256,3,padding=0,stride=1)
        
        
    def forward(self,inputdata):
        print("input size",inputdata.shape)
        a=F.relu(self.conv1_1(inputdata))
        a=F.relu(self.conv1_2(a))
        a=self.pool1(a)
        print("1:",a.shape)
        
        a=F.relu(self.conv2_1(a))
        a=F.relu(self.conv2_2(a))
        a=self.pool2(a)
        print("2:",a.shape)
        
        a=F.relu(self.conv3_1(a))
        a=F.relu(self.conv3_2(a))
        a=F.relu(self.conv3_3(a))
        a=self.pool3(a)
        conv3_features=a
        print("3:",a.shape)
        
        a=F.relu(self.conv4_1(a))
        a=F.relu(self.conv4_2(a))
        a=F.relu(self.conv4_3(a))
        print("4_3:",a.shape)
        a=self.pool4(a)
        conv4_features=a
        
        a=F.relu(self.conv5_1(a))
        a=F.relu(self.conv5_2(a))
        a=F.relu(self.conv5_3(a))
        a=self.pool5(a)
        print("5:",a.shape)
        conv4_features=a
        
        a=F.relu(self.conv6(a))
        a=F.relu(self.conv7(a))
        print("7:",a.shape)
        
        a=F.relu(self.conv8_1(a))
        a=F.relu(self.conv8_2(a))
        print("8:",a.shape)
        
        a=F.relu(self.conv9_1(a))
        a=F.relu(self.conv9_2(a))
        print("9:",a.shape)
        
        a=F.relu(self.conv10_1(a))
        a=F.relu(self.conv10_2(a))
        print("10:",a.shape)
        
        a=F.relu(self.conv11_1(a))
        a=F.relu(self.conv11_2(a))
        print("11:",a.shape)
        
#         a=F.relu(self.conv5_1(a))
#         a=F.relu(self.conv5_2(a))
#         a=F.relu(self.conv5_3(a))
#         a=self.pool5(a)
#         conv5_features=a
#         print("5:",a.shape)
        return a
    
Model=SSD()
Model._init_()
print(Model.forward(input_image).shape)

input size torch.Size([1, 3, 300, 300])
1: torch.Size([1, 64, 150, 150])
2: torch.Size([1, 128, 75, 75])
3: torch.Size([1, 256, 38, 38])
4_3: torch.Size([1, 512, 38, 38])
5: torch.Size([1, 512, 19, 19])
7: torch.Size([1, 1024, 19, 19])
8: torch.Size([1, 512, 10, 10])
9: torch.Size([1, 256, 5, 5])
10: torch.Size([1, 256, 3, 3])
11: torch.Size([1, 256, 1, 1])
torch.Size([1, 256, 1, 1])
